# Lesson  5 Serving Your Personal Tutor with a RESTful API Using FastAPI

Certainly! Here’s your content **converted into structured, readable Markdown** with code blocks and clear headings:

---

# Serving Your Personal Tutor with a RESTful API Using FastAPI

Welcome to the next step in our journey of building a personal tutor service with FastAPI. In the previous lesson, we focused on the `TutorController`, which manages tutoring sessions and handles student queries by interacting with both the model and service layers. Now, we will take a significant step forward by creating a RESTful API for our personal tutor service using **FastAPI**. We'll start by setting up the main FastAPI application, then adapt the `TutorController` to integrate with FastAPI's session management.

---

## Understanding RESTful APIs and FastAPI

RESTful APIs allow different software systems to communicate over the internet using a set of rules for exchanging data. **FastAPI** is a modern, fast (high-performance) web framework for building APIs with Python 3.7+, leveraging Python type hints and providing automatic, interactive API documentation.

### Install FastAPI and Uvicorn

To get started, install FastAPI and Uvicorn with pip:

```bash
pip install fastapi
pip install uvicorn
```

---

## Initializing a FastAPI App

Start by creating your FastAPI app:

```python
from fastapi import FastAPI

# Initialize FastAPI app
app = FastAPI(title="Personal Tutor API")
```

---

## Adding Session Management with Starlette Middleware

FastAPI is built on Starlette, which includes session middleware:

```python
from starlette.middleware.sessions import SessionMiddleware

# Add session middleware
app.add_middleware(
    SessionMiddleware,
    secret_key="your_secret_key_here"
)
```

The `SessionMiddleware` uses cookies to securely manage session data.

---

## Setting Up Static Files and Templates

Serve static files and render HTML templates for a user interface:

```python
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates

# Mount static files directory
app.mount("/static", StaticFiles(directory="static"), name="static")

# Setup Jinja2 templates
templates = Jinja2Templates(directory="templates")
```

---

## Creating the Controller Instance

Instantiate your `TutorController` for handling tutoring operations:

```python
from controllers.tutor_controller import TutorController

# Create an instance of TutorController
tutor_controller = TutorController()
```

---

## Defining Request Models with Pydantic

Use Pydantic to define and validate incoming API requests:

```python
from pydantic import BaseModel

class QueryRequest(BaseModel):
    session_id: str
    query: str
```

---

## Managing Student Sessions

Ensure every student has a unique identifier:

```python
import uuid
from fastapi import Depends, Request

async def get_student_id(request: Request):
    if "student_id" not in request.session:
        request.session["student_id"] = str(uuid.uuid4())
    return request.session["student_id"]
```

---

## Creating the Index Route

Render your homepage with a student session:

```python
@app.get("/")
async def index(request: Request, student_id: str = Depends(get_student_id)):
    return templates.TemplateResponse(
        "tutor.html", 
        {"request": request}
    )
```

---

## Creating New Tutoring Sessions

Define a route to create a new tutoring session:

```python
from fastapi import HTTPException

@app.post("/api/create_session")
async def create_session(student_id: str = Depends(get_student_id)):
    response = tutor_controller.create_session(student_id)
    
    if isinstance(response, tuple):
        error_data = response[0]["error"]
        raise HTTPException(
            status_code=error_data["code"],
            detail=error_data["message"]
        )
    
    return response["data"]
```

---

## Handling Student Queries

Define a route for students to send queries:

```python
@app.post("/api/send_query")
async def send_query(
    query_request: QueryRequest,
    student_id: str = Depends(get_student_id)
):
    response = tutor_controller.send_query(
        student_id,
        query_request.session_id,
        query_request.query
    )
    
    if isinstance(response, tuple):
        error_data = response[0]["error"]
        raise HTTPException(
            status_code=error_data["code"],
            detail=error_data["message"]
        )
    
    return response["data"]
```

---

## Custom Exception Handling

Provide consistent error responses using a custom handler:

```python
from fastapi.responses import JSONResponse

@app.exception_handler(HTTPException)
async def http_exception_handler(request: Request, exc: HTTPException):
    return JSONResponse(
        status_code=exc.status_code,
        content={
            "status": "error",
            "error": {
                "message": exc.detail,
                "code": exc.status_code
            }
        }
    )
```

---

## Running the FastAPI Server

Use Uvicorn to run your application:

```python
import uvicorn

if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )
```

* **host="0.0.0.0"**: Accepts connections from any network interface.
* **port=3000**: Runs on port 3000.
* **reload=True**: Enables hot-reloading during development.

---

## Adapting the TutorController

Your `TutorController` integrates smoothly with FastAPI session management and request handling.

```python
class TutorController:
    def __init__(self):
        self.tutor_service = TutorService()
    
    def create_session(self, student_id):
        if not student_id:
            return self._error_response('Session expired', 401)
        
        session_id = self.tutor_service.create_session(student_id)
        return self._success_response({
            'session_id': session_id,
            'message': 'Tutoring session created successfully'
        })
    
    def send_query(self, student_id, session_id, student_query):
        if not student_id:
            return self._error_response('Session expired', 401)
        
        if not session_id or not student_query:
            return self._error_response('Missing session_id or query', 400)
            
        try:
            tutor_response = self.tutor_service.process_query(
                student_id, 
                session_id, 
                student_query
            )
            return self._success_response({'message': tutor_response})
        except ValueError as e:
            return self._error_response(str(e), 404)
        except RuntimeError as e:
            return self._error_response(str(e), 500)
    
    def _success_response(self, data):
        response = {
            'status': 'success',
            'data': data
        }
        return response
    
    def _error_response(self, message, status_code):
        response = {
            'status': 'error',
            'error': {
                'message': message,
                'code': status_code
            }
        }
        return response, status_code
```

---

## How Students Interact with the Personal Tutor API

1. **Access the Index Route (`/`)**
   Establishes a student session and loads the tutor interface.

2. **Create a Tutoring Session (`/api/create_session`)**
   Returns a unique session ID for the student's tutoring session.

3. **Send a Query (`/api/send_query`)**
   Student sends academic questions; receives AI-generated tutor responses.

---

## Summary and Next Steps

In this lesson, we built a RESTful API for our personal tutor service using FastAPI.
We set up the FastAPI app, defined routes for tutoring operations, and connected everything using session management and controller integration.

As you move on to the practice exercises, experiment with the FastAPI API to reinforce these concepts.
This hands-on work will prepare you for deeper development and additional FastAPI and RESTful API features.

**Keep up the great work, and I look forward to seeing your progress!**


## Serving Your Personal Tutor with a RESTful API Using FastAPI

In this exercise, you'll apply your knowledge of FastAPI to create a simple RESTful API. Your task is to:

Initialize a FastAPI app and reference it as "main:app".
Define an index route at "/" that returns a welcome message.
Set up the app to run on port 3000 and host '0.0.0.0' with reload=True using "main:app" as the application reference.
This exercise is designed to help you build a basic FastAPI application that simply returns a message. At this stage, we are not involving any other layers of our app. Dive in and see how straightforward it can be!

```python
from fastapi import FastAPI
import uvicorn

# TODO: Initialize the FastAPI application


# TODO: Define a route for the index page at "/"
# The route should return "Welcome to the Personal Tutor Service!"

# TODO: Run the FastAPI application with Uvicorn
# - Use "main:app" as the application reference
# - Set host to '0.0.0.0'
# - Set port to 3000
# - Enable reload with reload=True

```

Here’s the complete code for your **basic FastAPI RESTful API exercise**:

```python
from fastapi import FastAPI
import uvicorn

# Initialize the FastAPI application
app = FastAPI()

# Define a route for the index page at "/"
@app.get("/")
async def index():
    return {"message": "Welcome to the Personal Tutor Service!"}

# Run the FastAPI application with Uvicorn
if __name__ == "__main__":
    uvicorn.run(
        "main:app",   # "main" refers to the filename, "app" is the FastAPI instance
        host="0.0.0.0",
        port=3000,
        reload=True
    )
```

**How to run:**

* Save this code in a file named `main.py`.
* Run it using:

  ```bash
  python main.py
  ```
* Open [http://localhost:3000](http://localhost:3000) and you should see your welcome message!

Let me know if you want to add more routes or functionality!


## Building a Simple FastAPI Welcome API

## Implementing Session Management in FastAPI TutorController

## Implementing a Tutoring Session Creation Route in FastAPI